## Setup


In [1]:
import requests
import json
import base64
import brikasutils as bu
import test_utils as tu
import random
from aiohttp import ClientTimeout  # noqa: F401
from dotenv import load_dotenv
from typing import Dict, List
import os
from pathlib import Path
from aiohttp import FormData

from faker import Faker
fake = Faker()
Faker.seed("ITEMS")

import importlib
importlib.reload(bu)
importlib.reload(tu)

from brikasutils import RequestTemplate as RT

load_dotenv(override=True)
ADMIN_API_KEY = os.getenv('ADMIN_API_KEY') or 'undefined'

HEADERS_CONTENT_JSON = {'Content-Type': 'application/json'}

# Endpoint Base URLs - DEVELOPMENT - FastAPI
# BASE_URL_AUTH = 'http://91.99.7.176:8000/auth'
# BASE_URL_FUNCTIONS = 'http://91.99.7.176:8000'
BASE_URL_AUTH = 'http://localhost:8000/auth'
BASE_URL_FUNCTIONS = 'http://localhost:8000'

### Hello World


In [ ]:
f_endpoint = f'{BASE_URL_FUNCTIONS}/test/demo'
params = {
    'name': 'John Doe',
}
response = requests.get(f_endpoint, params=params, timeout=10)
print(response.status_code)
print(response.text)


## Users


### Setup Users


In [ ]:
MIME_TYPE = 'image/jpeg'

IMAGE_SQ_PERSON_1 = 'img/test-img-sq-person-1.jpg'
IMAGE_SQ_PERSON_2 = 'img/test-img-sq-person-2.jpg'
IMAGE_SQ_PERSON_3 = 'img/test-img-sq-person-3.jpg'
IMAGE_SQ_PERSON_4 = 'img/test-img-sq-person-4.jpg'
IMAGE_NONSQ_PERSON_1 = 'img/test-img-nonsq-person-1.jpg'
IMAGE_NONSQ_PERSON_2 = 'img/test-img-nonsq-person-2.jpg'

IMAGE_SQ_PERSON_5 = 'img_local/user_elias.jpg'
IMAGE_SQ_PERSON_6 = 'img_local/user_patryk.jpg'
IMAGE_SQ_PERSON_7 = 'img_local/user_petyo.jpg'


IMAGE_SQ_GROUP_DORM = 'img/test-img-sq-group-dorm.jpg'
IMAGE_SQ_GROUP_DTU = 'img/test-img-sq-group-dtu.jpg'
IMAGE_SQ_GROUP_LOGO = 'img/test-img-sq-group-logo.jpg'


users = [
    {
        "email": "airidas.brikas@gmail.com",
        "password": "1234567",
        "first_name": "Airidas",
        "last_name": "Brikas",
        "send_welcome_email": False,
        "image": IMAGE_SQ_PERSON_1,
        # "token": str
        # "id": str
        # "groups": ["..."]

    },
    {
        "email": "patryk@t.com",
        "password": "1234567",
        "first_name": "Patryk",
        "last_name": "Krynojeswki",
        "image": IMAGE_SQ_PERSON_6,
    },
    {
        "email": "anna@t.com",
        "password": "1234567",
        "first_name": "Anna",
        "last_name": "Kowalska",
        "image": IMAGE_SQ_PERSON_2,
    },
    {
        "email": "petyo@t.com",
        "password": "1234567",
        "first_name": "Petyo",
        "last_name": "Petkov",
        "image": IMAGE_SQ_PERSON_7
        # No image 
    },
    {
        "email": "david@t.com",
        "password": "1234567",
        "first_name": "David",
        "last_name": "Jaic",
        "image": IMAGE_SQ_PERSON_3,
    },
    {
        "email": "elias@t.com",
        "password": "1234567",
        "first_name": "Elias",
        "last_name": "Torjani",
        "image": IMAGE_SQ_PERSON_5,
    },
    # {
    #     "phone_number": "+4560905504",
    #     "first_name": "Airidas",
    #     "last_name": "Denmark",
    #     "image": IMAGE_SQ_PERSON_3,
    # },
    # {
    #     "phone_number": "+37067320686",
    #     "first_name": "Airidas",
    #     "last_name": "Lithuania",
    #     "image": IMAGE_SQ_PERSON_2,
    # },
    # {
    #     "phone_number": "+4527144116",
    #     "first_name": "Petyo",
    #     "last_name": "Phone Num",
    #     "image": IMAGE_SQ_PERSON_2,
    # },


]

# Apply defaults
def apply_user_defaults():
    for user in users:
        if "send_welcome_email" not in user:
            user["send_welcome_email"] = False
        if "image" not in user:
            user["image"] = None
apply_user_defaults()

### Add additional, special users


In [ ]:
# Add new users
add_users = [
    # EXTERNALLY INVITED LATER 
    {
        "email": "ninjatxgaming@gmail.com",
        "password": "1234567",
        "first_name": "Airidas",
        "last_name": "Brikas2",
        # "image": image,
    },
]

for user in add_users:
    if not any(u['email'] == user['email'] for u in users):
        users.append(user)
    else:
        for u in users:
            if u['email'] == user['email']:
                # Delete the user from the list
                users.remove(u)
                users.append(user)
                print(f"User {user['email']} replaced")
                break


apply_user_defaults()


##### _Add additional generated users_


In [ ]:
# Generate X amount of groups using faker
NUM_USERS = 150  # Set the number of groups you want to generate
BATCH_ID = "gen-users-1"

add_users = []
for i in range(NUM_USERS):
    add_users.append({
        "email": fake.email(),
        "password": "1234567",
        "first_name": fake.first_name(),
        "last_name": fake.last_name(),
        "batch_id": BATCH_ID,
    })

# Add generated groups to the main groups list and apply defaults
users.extend(add_users)
apply_user_defaults()
print(f"Added {NUM_USERS} generated users.")

### Register New users


In [ ]:
ONLY_NON_EXISTING = True

SEND_EMAIL = False
rts = []
for user in users:
    if ONLY_NON_EXISTING and 'id' in user and user["id"] != None:
        continue

    def callback(response: bu.CustomResponse, user=user):        
        if response.status == 200:
            print(f"{response.status}: {user['email']}")
            r = response.json()
            if r != None:
                user['token'] = r["access_token"]
                user['id'] = r["user_id"]
        else:
            print(f"{response.status}: {user['email']}, response: {response.text}")

    rts.append(
        RT(
            method="post",
            url=f'{BASE_URL_AUTH}/register',
            json={
                "email": user['email'],
                "password": user['password'],
                "send_email": True if user['send_welcome_email'] else SEND_EMAIL,
                "test_id": user['batch_id'] if 'batch_id' in user else None,
            },
            timeout=60,
            headers=HEADERS_CONTENT_JSON,
            callback=callback,
        )
    )

responses = bu.run_async_requests(rts, verbose=False, max_concurrent=5)

### Log in Users


In [ ]:
ONLY_NON_LOGGED_IN = False
AUTH_ENDPOINT = f'{BASE_URL_AUTH}/token'

rts = []

for user in users:
    if ONLY_NON_LOGGED_IN and 'token' in user and user['token'] != None:
        continue

    if "email" in user and "password" in user:
        def callback(response: bu.CustomResponse, user=user):
            if response.status == 200:
                print(f"{response.status}: {user['email']}")
                r = response.json()
                if r != None:
                    user['token'] = r["access_token"]
                    user['id'] = r["user_id"]
            else:
                print(f"{response.status}: {user['email']}, response: {response.text}")

        rts.append(
            RT(
                method="post",
                url=AUTH_ENDPOINT,
                json={
                    "email": user['email'],
                    "password": user['password'],
                },
                timeout=30,
                headers=HEADERS_CONTENT_JSON,
                callback=callback
            )
        )

responses = bu.run_async_requests(rts, verbose=False)



### Test Login


In [ ]:
# Test /me endpoint
rts = []
for user in users:
    if "token" not in user or user['token'] == None:
        print(f"User {user['email']} does not have token, skipping /me test")
        continue

    def callback(response: bu.CustomResponse, user=user):
        if response.status == 200:
            r = response.json()
            if r == None:
                print(f"{response.status}: /me {user['email']}, response: None")
                return
            print(f"{response.status}: /me {user['email']} -> id: {r['id']}")
        else:
            print(f"{response.status}: /me {user['email']}, response: {response.text}")

    rts.append(
        RT(
            method="get",
            url=f'{BASE_URL_AUTH}/users/me',
            timeout=10,
            headers={
                'Authorization': f'Bearer {user["token"]}'
            },
            callback=callback
        )
    )
responses = bu.run_async_requests(rts, verbose=False)

### Update User


In [ ]:
only_users = [
    # tu.get_user_from_email("petyo@t.com", users)['id'],
]

rts = []
for user in users:

    if len(only_users) > 0 and user['id'] not in only_users:
        continue

    def callback(response: bu.CustomResponse, user=user):
        print(f"{response.status}: {tu.emailOrPhone(user)}, response: {response.text}")


    rts.append(
        RT(
            method="post",
            url=f'{BASE_URL_FUNCTIONS}/users/{user["id"]}',
            json={
                "first_name": user['first_name'],
                "last_name": user['last_name'],
                # "image": tu.encode_base64_image_or_none(user['image']),
            },
            timeout=30,
            headers= tu.auth_json_headers(user['token']),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False, max_concurrent=10)

### Update Profile Image


In [ ]:
user_image_updates = [
    {
        "action_caller": tu.get_user_from_email("airidas.brikas@gmail.com", users),
        "image": "img_local/user_airidas_2.jpg",
    }
]

# Add all users with images to the list
for user in users:
    if 'image' in user and user['image'] is not None:
        user_image_updates.append({
            "action_caller": user,
            "image": user['image'],
        })

rts = []

for update in user_image_updates:
    user = update['action_caller']

    form = FormData()
    form.add_field(
        'image',
        open(update['image'], 'rb'),
        filename=Path(update['image']).name,
        content_type="image/jpeg" 
    )

    def callback(response: bu.CustomResponse, user=user):
        if response.status == 200:
            r = response.json()
            if r != None:
                print(f"{response.status}: {tu.emailOrPhone(user)}", 
                      f"image_url: {r['image_url']}")
            else:
                print(f"{response.status}: {tu.emailOrPhone(user)}, no response data")
        else:
            print(f"{response.status}: {tu.emailOrPhone(user)}, response: {response.text}")
    
    rts.append(
        RT(
            method="post",
            url=f'{BASE_URL_FUNCTIONS}/users/{user["id"]}/profile-image',
            data=form,
            timeout=30,
            headers={
                "Authorization": f"Bearer {user['token']}",
            },
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False, max_concurrent=3)

### Change Password


In [ ]:
queries = [
    {
        "action_caller": tu.get_user_from_email("airidas.brikas@gmail.com", users),
        "current_password": "1234567890",
        "new_password": "1234567",
    },
    {
        "action_caller": tu.get_user_from_email("patryk@t.com", users),
        "current_password": "1234567",
        "new_password": "12345678",
    }
]

rts = []
for query in queries:
    user = query['action_caller']

    def callback(response: bu.CustomResponse, user=user, query=query):
        if response.status == 200:
            r = response.json()
            print(f"{response.status}: {tu.emailOrPhone(user)}. Updating local records.")
            user['password'] = query['new_password']
        else:
            print(f"{response.status}: {tu.emailOrPhone(user)}, response: {response.text}")
    
    rts.append(
        RT(
            method="post",
            url=f'{BASE_URL_FUNCTIONS}/users/me/change-password',
            json={
                "current_password": query['current_password'],
                "new_password": query['new_password'],
            },
            timeout=30,
            headers= tu.auth_json_headers(user['token']),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

### Search Users


In [ ]:
queries = [
    {
        "action_caller": { "email": "david@t.com"},
        "query": "pat",
    }
]

rts = []
for query in queries:
    user = tu.get_user_from_email(query['action_caller']['email'], users)

    def callback(response: bu.CustomResponse, user=user):
        if response.status_code == 200:
            r = response.json()
            if r != None:
                print(f"{response.status}: {user['email']} -> {len(r['results'])} results")
                for res in r['results']:
                    print(f" - {res['first_name']} {res['last_name']} ({res['email']})")
        else:
            print(f"{response.status}: {user['email']}, response: {response.text}")

    rts.append(
        RT(
            method="get",
            url=f'{BASE_URL_FUNCTIONS}/users',
            params={
                "q": query["query"],
            },
            timeout=10,
            headers=tu.auth_json_headers(user["token"]),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

### Get User


In [ ]:
queries = [
    {
        "action_caller_id": tu.get_user_from_email("patryk@t.com", users)['id'],
        "user_id": tu.get_user_from_email("airidas.brikas@gmail.com", users)["id"],
    },
]

rts = []
for query in queries:
    user = tu.get_user_from_id(query['action_caller_id'], users)

    def callback(response: bu.CustomResponse, user=user, query=query):
        if response.status_code == 200:
            r = response.json()
            if r != None:
                print(f"{response.status}: {user['email']} -> {query["user_id"]},")
                print(json.dumps(r, indent=2))
        else:
            print(f"{response.status}: {user['email']} -> {query["user_id"]}, response: {response.text}")

    rts.append(
        RT(
            method="get",
            url=f'{BASE_URL_FUNCTIONS}/users/{query["user_id"]}',
            timeout=10,
            headers=tu.auth_json_headers(user["token"]),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

### Delete User


In [ ]:
# Delete user account
queries = [
    {
        "action_caller": tu.get_user_from_email("patryk@t.com", users),
    }
]

rts = []
for query in queries:
    user = query['action_caller']

    def callback(response: bu.CustomResponse, user=user):
        if response.status_code == 200:
            print(f"{response.status}: {user['email']} - Account deleted successfully")
        else:
            print(f"{response.status}: {user['email']}, response: {response.text}")

    rts.append(
        RT(
            method="delete",
            url=f'{BASE_URL_FUNCTIONS}/users/me',
            timeout=10,
            headers=tu.auth_json_headers(user["token"]),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)